# ITRA-DT APP

# #Code start

For beggining we need to import python modules for our application to work. You must know this: there are multiple ways, to code with Kivy GUI and OpenCV. In this version, you will only see Python based code. At the end of this document you will see an alternative way of using this application, since you can also use Builder sub-module.

In [ ]:
#Main modules for this application:
import os, sys, clock
import webbrowser
from typing import Union
from time import time
from datetime import datetime
from os.path import dirname, join
import glob

#OpenCV module
import cv2

#Numpy module
import numpy as np

#Kivy widgets
from kivymd.app import MDApp
from kivymd.uix.boxlayout import MDBoxLayout
from kivymd.uix.button import MDRaisedButton
from kivymd.uix.toolbar import MDTopAppBar
from kivymd.uix.bottomnavigation import MDBottomNavigation, MDBottomNavigationItem
from kivy.uix.image import Image
from kivy.core.window import Window
from kivymd.uix.filemanager import MDFileManager
from kivymd.toast import toast
from kivy.properties import ObjectProperty, NumericProperty, StringProperty, BooleanProperty,\
    ListProperty
from kivy.graphics.texture import Texture
from kivy.clock import Clock


Main application starts. We create python class that will inherit KivyMD Application module classes. We added title, but we can also create icon, that will be displayed in window.

In [ ]:
class MainApp(MDApp):
    title = 'IDRA-DT'
    icon = ''
    
    #Here we will create __init__ function, so that the apps window will be bined to file manager
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        Window.bind(on_keyboard=self.events)
        self.manager_open = False
        self.file_manager = MDFileManager(
            exit_manager=self.exit_manager, select_path=self.select_path
        )
        
    #Here we will continue with native build function in KivyMD application module
    def build(self):
        self.theme_cls.theme_style = "Dark"
        self.theme_cls.primary_palette = "Amber"
        self.theme_cls.material_style = "M3"
        main_layout = MDBoxLayout(orientation = 'vertical')
        menu = MDTopAppBar(id = 'toolbar',
                            use_overflow = True,
                            pos_hint = {"top": 1},
                            size_hint_x = 1,
                            left_action_items =  [["menu", lambda x: self.callback(x)],["file", lambda x: self.file_manager_open()]],
                            right_action_items =  [["tools", lambda x: self.display_settings()],["information", lambda x: self.app_timer()]],
                            md_bg_color =  self.theme_cls.primary_color
                            )


        bottom = MDBottomNavigation()



        #START OF CAMERA SCREEN
        item_camera = MDBottomNavigationItem(id = 'cameras',
                                            name = 'Cameras',
                                            text = "Camera Settings",
                                            icon = 'camera')

        cam_layout = MDBoxLayout(orientation = 'vertical',
                                    pos_hint = {"center_x": 0.5, "center_y": 0.5},
                                    size_hint_y = 0.8,
                                    )

        self.image = Image()
        # self.image.size_hint_y = None
        # self.image.height = 200
        self.image.allow_stretch = True
        cam_layout.add_widget(self.image)
        cam_layout.add_widget(MDRaisedButton(
            text = 'Low T template',
            pos_hint = {'center':.5 , 'center_y':.5},
            size_hint = (None,None),
            on_press = self.take_lowtemplate
        ))
        cam_layout.add_widget(MDRaisedButton(
            text = 'High T template',
            pos_hint = {'center':.5 , 'center_y':.5},
            size_hint = (None,None),
            on_press = self.take_hightemplate
        ))
        self.capture = cv2.VideoCapture(2)
        Clock.schedule_interval(self.load_video,1.0/30.0)
        item_camera.add_widget(cam_layout)

        #START OF MACHINE VISION ALGORITHMS
        item_vision = MDBottomNavigationItem(id = 'visions',
                                            name = 'Visions',
                                            text = "Machine Vision Settings",
                                            icon = 'camera-control')


        visions_layout = MDBoxLayout(orientation = 'vertical',
                                    pos_hint = {"center_x": 0.5, "center_y": 0.5},
                                    size_hint_y = 0.8,
                                    )
        self.template = Image()

        item_vision.add_widget(visions_layout)


        #bottom items together

        bottom.add_widget(item_camera)
        bottom.add_widget(item_vision)

        #MAIN WIDGETS
        main_layout.add_widget(menu)
        main_layout.add_widget(bottom)
        return main_layout
    
    #Here we can add settings function, that are native, but be reminded! You must also create a menu list
    # or you must create any other widget that has a callable function. Then you must code settings here in this function
    # It is better to use callable class
    def display_settings(settings):
        return True
    #this is a function for selecting files in filemanager
    def selected(self, filename):
        try:
            self.ids.img.source = filename[0]
        except:
            pass

    #this is main video function to display video in application. There is also an alternative way using native Kivy Camera module that is also coded in OpenCV.
    def load_video(self, *args):
        gray = self.capture.read(cv2.IMREAD_GRAYSCALE)
        alpha = self.capture.read(cv2.IMREAD_UNCHANGED)
        depth = self.capture.read(cv2.IMREAD_ANYDEPTH)
        

        infern = cv2.applyColorMap(gray, cv2.COLORMAP_INFERNO)
        jetp = cv2.applyColorMap(infern, cv2.COLORMAP_JET)
        #virdis = cv2.applyColorMap(jetp, cv2.COLORMAP_VIRIDIS)
        invert = cv2.bitwise_not(jetp)

        frame = invert
        self.image_frame = frame
        buffer = cv2.flip(frame, 0).tobytes() # tostring is deprecated
        texture = Texture.create(size=(frame.shape[1], frame.shape[0]))
        texture.blit_buffer(buffer, bufferfmt='ubyte')
        self.image.texture = texture

    def take_lowtemplate(self, *args):
        image_name = "tmpl/template_low.png"
        
        cv2.imwrite(image_name, self.image_frame)
    def take_hightemplate(self, *args):
        image_name = "tmph/template_high.png"
        cv2.imwrite(image_name, self.image_frame)

    def on_save(self, instance, value):
        print(instance, value)

    def on_cancel(self, instance, value):
        print(instance, value)
    def callback(self, button):
        self.menu.caller = button
        self.menu.open()

    def home_callback(self, text_item):
        self.menu.dismiss()
        Snackbar(text=text_item).open()

    def menu_callback(self, text_item):
        self.menu.dismiss()
        Snackbar(text=text_item).open()


    def on_pause(self):
        return True

    def on_resume(self):
        pass

    #IMAGE LOAD command
    def load_image(self, *args):
        self.cv2.imread()

    #HELP commands
    def help(self):
        webbrowser.open('https://docs.opencv.org/4.x/')
    #FILE MANAGER COMMANDS
    def file_manager_open(self):
        self.file_manager.show(os.path.expanduser("~"))  # output manager to the screen
        self.manager_open = True

    def select_path(self, path: str):
        '''
        It will be called when you click on the file name
        or the catalog selection button.

        :param path: path to the selected directory or file;
        '''

        self.exit_manager()
        toast(path)

    def exit_manager(self, *args):
        '''Called when the user reaches the root of the directory tree.'''

        self.manager_open = False
        self.file_manager.close()

    def events(self, instance, keyboard, keycode, text, modifiers):
        '''Called when buttons are pressed on the mobile device.'''

        if keyboard in (1001, 27):
            if self.manager_open:
                self.file_manager.back()
        return True
    
if __name__ == '__main__':
    MainApp().run()

This app only runs and displays applied colors on grayscale image. It is still in development, but it will work, since temperature templates can be calculated and displayed by using vector matrixes on each RGB channel. It is more explained in next chapter.

# #Thermal Image processing - fake thermal image processing

Here we will explain main load_video function. Cameras as webcams (internal, external), and others ... they all use CMOS sensors, well that really depends from manufacturer and brand (Sony, Canon, Logitech, Canyon...ect..). These images, that are captured through normal RGB channels, are often displayed as 8bit images, for what means that each channel has only from 0 to 255 values. And they are only 3 of these channels: RED, GREEN, BLUE. As an alternative, many computer vision systems use also HSV method of image processing. But in case of thermal image processing you must now this: images captured using infrared or thermal vision camera, have all 16bit values, wich means that each channel has 65535 values. 

To fake thermal image, we must understand 16bit image. 8bit image can be converted to 16bit by using software as GNU GIMP. But it is not neccessary that will work, since thermal image also contains lumination (lux) wavelenght on each pixel. To create fake thermal image, we need to do this: capture image with ANYDEPTH method (this gives in-pixel depth information, but from 0 - 255 values on each channel), we need to capture raw image (an alpha image - image that is not changed, at least for now no gamma corrections and stuff), and normal GRAYSCALE image.

In [ ]:
    def load_video(self, *args):
        gray = self.capture.read(cv2.IMREAD_GRAYSCALE)
        alpha = self.capture.read(cv2.IMREAD_UNCHANGED)
        depth = self.capture.read(cv2.IMREAD_ANYDEPTH)

Above you can see three types of captures. Now to convert it to 16bit image, we need to add this code or correct it

In [ ]:
    def load_video(self, *args):
        gray = self.capture.read(cv2.IMREAD_GRAYSCALE)
        alpha = self.capture.read(cv2.IMREAD_UNCHANGED)
        
        
        depth = self.capture.read(cv2.IMREAD_ANYDEPTH)
        #change depth to grayscale
        grayed = cv2.cvtColor(dept, )

In [ ]:

        infern = cv2.applyColorMap(gray, cv2.COLORMAP_INFERNO)
        jetp = cv2.applyColorMap(infern, cv2.COLORMAP_JET)
        #virdis = cv2.applyColorMap(jetp, cv2.COLORMAP_VIRIDIS)
        invert = cv2.bitwise_not(jetp)

        frame = invert
        self.image_frame = frame
        buffer = cv2.flip(frame, 0).tobytes() # tostring is deprecated
        texture = Texture.create(size=(frame.shape[1], frame.shape[0]))
        texture.blit_buffer(buffer, bufferfmt='ubyte')
        self.image.texture = texture